# Based on EDWARD CROOKENDEN's Notebook 

Notebooks link: https://www.kaggle.com/code/edwardcrookenden/otto-getting-started-eda-baseline

This one is not completed. Most of the works are for me to learn from start. 

In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from pathlib import Path
import pandas as pd
import random
import numpy as np
import json
from datetime import timedelta
from collections import Counter
from tqdm.notebook import tqdm
from heapq import nlargest

import warnings
warnings.filterwarnings('ignore')
from collections import Counter
from random import sample

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/otto-recommender-system/sample_submission.csv
/kaggle/input/otto-recommender-system/test.jsonl
/kaggle/input/otto-recommender-system/train.jsonl


In [75]:
### Paths ###

DATA_PATH = Path('../input/otto-recommender-system')
TRAIN_PATH = DATA_PATH/'train.jsonl'
TEST_PATH = DATA_PATH/'test.jsonl'
SAMPLE_SUB_PATH = Path('../input/otto-recommender-system/sample_submission.csv')

## EDA and baseline

In [76]:
# check on the train data
# with open(TRAIN_PATH, 'r') as f:
#     print(f"We have {len(f.readlines()):,} lines in the training data")
    
# Too many lines, Not gonna run it.
# We have 12,899,779 lines in the training data

In [77]:
# read jsonl
sample_size = 1000

chunks = pd.read_json(TRAIN_PATH, lines=True, chunksize = sample_size)

for c in chunks:
    sample_train_df = c
    break

#session id starts from 0

In [78]:
#Rename the id column with session number, session id is useful and can be changed
sample_train_df.set_index('session', drop=True, inplace=True)
sample_train_df.head()

,events
session,
0,"[{'aid': 1517085, 'ts': 1659304800025, 'type':..."
1,"[{'aid': 424964, 'ts': 1659304800025, 'type': ..."
2,"[{'aid': 763743, 'ts': 1659304800038, 'type': ..."
3,"[{'aid': 1425967, 'ts': 1659304800095, 'type':..."
4,"[{'aid': 613619, 'ts': 1659304800119, 'type': ..."


In [79]:
#events is the only column in the df
sample_train_df.columns

Index(['events'], dtype='object')

In [80]:
# Events IS A LIST
# Inside Events is a dict
len(sample_train_df['events'][0])
type(sample_train_df['events'][0][0])

dict

In [81]:
session0 = sample_train_df['events'][0]
#276 items

In [82]:
# sample_train_df['events'][0]

[{'aid': 1517085, 'ts': 1659304800025, 'type': 'clicks'},
 {'aid': 1563459, 'ts': 1659304904511, 'type': 'clicks'},
 {'aid': 1309446, 'ts': 1659367439426, 'type': 'clicks'},
 {'aid': 16246, 'ts': 1659367719997, 'type': 'clicks'},
 {'aid': 1781822, 'ts': 1659367871344, 'type': 'clicks'},
 {'aid': 1152674, 'ts': 1659367885796, 'type': 'clicks'},
 {'aid': 1649869, 'ts': 1659369893840, 'type': 'carts'},
 {'aid': 461689, 'ts': 1659369898050, 'type': 'carts'},
 {'aid': 305831, 'ts': 1659370027105, 'type': 'orders'},
 {'aid': 461689, 'ts': 1659370027105, 'type': 'orders'},
 {'aid': 362233, 'ts': 1659370064916, 'type': 'clicks'},
 {'aid': 1649869, 'ts': 1659370067686, 'type': 'clicks'},
 {'aid': 1649869, 'ts': 1659371003682, 'type': 'clicks'},
 {'aid': 984597, 'ts': 1659371033243, 'type': 'clicks'},
 {'aid': 1649869, 'ts': 1659371042297, 'type': 'clicks'},
 {'aid': 803544, 'ts': 1659371044075, 'type': 'clicks'},
 {'aid': 1110941, 'ts': 1659371104329, 'type': 'clicks'},
 {'aid': 1190046, 'ts': 

## Submission 1: only contains test data result in too many Missing values

We know that test data are truncated train data by time. It means that predictions I have now only captured part of the features. The result should be merged with ___?

## Submission on the test data scoring in 0.061
Update the missing values to the most frequenctly aid in the training dataset. 

In [83]:

i = 0
preds = []

while i< 3:

    session = sample_train_df['events'][i]
    #print(session[0])
    # split on type

    
    # Saved aids for each type
    click = []
    carts = []
    order = []
    for d in session:
        if d['type'] == 'clicks':
            click.append(d['aid'])
        if d['type'] == 'carts':
            carts.append(d['aid'])
        if d['type'] == 'orders':
            order.append(d['aid'])
    # next move addin counter
    a = Counter(click)
    top_click = nlargest(20, a, key = a.get)
    b = Counter(carts)
    top_carts = nlargest(20, b, key = b.get)
    c = Counter(order)
    top_order = nlargest(20, c, key = c.get)
    print(f'topclick:{top_click},top cart{top_carts}, top order{top_order}')
#  This is the full version for (str(id) for id in top_click)
#     str(id) for id in top_click:
#         print(str(id))
    
    

# Append three types preds in sequence 
    preds.append(" ".join([str(id) for id in top_click]))
    preds.append(" ".join([str(id) for id in top_carts]))
    preds.append(" ".join([str(id) for id in top_order]))
    i = i+1

topclick:[543308, 964169, 1072782, 1813509, 1436439, 1760145, 1549618, 362233, 1649869, 1110548, 784978, 1624436, 1157411, 1840615, 275288, 570955, 974651, 442293, 1349536, 984597],top cart[974651, 543308, 1649869, 461689, 789245, 280978, 1521766, 1760145, 275288, 442293, 1549618, 1199474, 315914], top order[305831, 461689, 1199474, 543308]
topclick:[1734061, 718983, 1492293, 910862, 1491172, 424964, 1515526, 440486, 109488, 1507622, 854637, 215311, 711125, 50049, 105393, 959544, 1842593, 1464360, 207905, 1628317],top cart[424964, 1492293, 910862, 1491172, 854637, 215311, 711125, 105393], top order[]
topclick:[137492, 795863, 935830, 1577398, 763743, 504789, 378348, 26638, 817441, 1405904, 545290, 1593105, 427698, 414004, 465360, 526287, 567119, 78519, 1605583, 690631],top cart[161269], top order[]


In [84]:
# Check on the lenghth, should be 3X3
len(preds)

9

In [68]:
# The results shows even in Train data there are not always 20 recommandations to have
# Need to figure out the source to fill in those blanks


['59625',
 '',
 '',
 '1142000 582732 973453 736515',
 '',
 '',
 '199008 141736 57315 194067 918667',
 '199008',
 '',
 '779477 1494780 476063 413962 562753 975116 595994 889671 229748 1669402 1674681 602722 1596098 45034 603159 1037537 1299062 1352725 1344773 794259',
 '779477 834354 1494780 413962 562753 476063 975116 595994 1344773 1711180 127404 889671 987399 740494',
 '1669402 829180 1696036 479970 834354 1033148 595994 1007613',
 '255297 300127 1114789 198385 1729553 1216820 1754419 607638 1817895',
 '',
 '',
 '1036375 1269952 476216 1579935 1546830 22981 1190477',
 '',
 '',
 '453905 258458 775584 41655 1497876 383003 1784451 95259 1308865 587969 160666 253080 1169631 1154962 543919 874493 1433061 1179870 614626 970225',
 '',
 '',
 '955252',
 '955252',
 '',
 '1682750 1024433',
 '1682750',
 '',
 '245131 39846 1259911 1663048',
 '',
 '',
 '525156 631398 1569899',
 '',
 '',
 '1830166',
 '1219653 1830166',
 '1219653',
 '915175 1501214 1365651 1787713 1305729',
 '',
 '',
 '1537160 16721

## Prep

In [39]:
# #Save 3 types in different lists
# click = []
# carts = []
# order = []
# for d in session0:
#     if d['type'] == 'clicks':
#         click.append(d['aid'])
#     if d['type'] == 'carts':
#         carts.append(d['aid'])
#     if d['type'] == 'orders':
#         order.append(d['aid'])


In [43]:
# #top 1 frequent
# from collections import Counter

# a = Counter(click)

 
# def most_frequent(List):
#     occurence_count = Counter(List)
#     return occurence_count.most_common(1)

# most_frequent(a)

[(543308, 7)]

In [46]:
# #top 20 frequent
# a = Counter(click)
# top_click_article = nlargest(20, a, key = a.get)

In [ ]:
# #timestamp 
# import datetime

# timestamp = "1659367439426"
# your_dt = datetime.datetime.fromtimestamp(int(timestamp)/1000)  # using the locauil timezone
# print(your_dt.strftime("%Y-%m-%d %H:%M:%S")) 

# Submission 2: fill the missing value with the most frequent Aid in training data

In [88]:
# Find out the top20 most frequent aids in each type
i = 0

click = []
carts = []
order = []
while i< 1000:

    session = sample_train_df['events'][i]
    #print(session[0])
    # split on type

    
    # Saved aids for each type

    for d in session:
        if d['type'] == 'clicks':
            click.append(d['aid'])
        if d['type'] == 'carts':
            carts.append(d['aid'])
        if d['type'] == 'orders':
            order.append(d['aid'])
    
    i+=1 
            

In [89]:
a_C = Counter(click)
top_click_C = nlargest(20, a_C, key = a_C.get)
b_C = Counter(carts)
top_carts_C = nlargest(20, b_C, key = b_C.get)
c_C = Counter(order)
top_order_C = nlargest(20, c_C, key = c_C.get)

In [90]:
top_click_C

[1588163,
 709803,
 448220,
 1603001,
 954951,
 1722991,
 159789,
 335674,
 1116095,
 1786822,
 1619041,
 1700164,
 196279,
 202774,
 602469,
 1083299,
 554660,
 1448648,
 432679,
 865245]

## Check on the submission file

In [115]:
sample_submission = pd.read_csv(SAMPLE_SUB_PATH)
sample_submission.head()

,session_type,labels
0,12899779_clicks,129004 126836 118524
1,12899779_carts,129004 126836 118524
2,12899779_orders,129004 126836 118524
3,12899780_clicks,129004 126836 118524
4,12899780_carts,129004 126836 118524


In [116]:
len(sample_submission)

5015409

In [117]:
sample_submission['session_type'].values[-1]

'14571581_orders'

## Applying on Test data

In [108]:
# read test data 1671803 data for this submission
sample_size = 1671803

chunks = pd.read_json(TEST_PATH, lines=True, chunksize = sample_size)

for c in chunks:
    sample_test_df = c
    break


In [113]:
sample_test_df['events'][1671802]

[{'aid': 1100210, 'ts': 1662328791563, 'type': 'clicks'}]

In [109]:
# Copy pasting . . .


i = 0
preds = []
# in total 1671803 rows
while i<  1671803:

    session = sample_test_df['events'][i]
 
    click = []
    carts = []
    order = []
    for d in session:
        if d['type'] == 'clicks':
            click.append(d['aid'])
        elif  d['type'] == 'carts':
            carts.append(d['aid'])
        elif  d['type'] == 'orders':
            order.append(d['aid'])
   
    a = Counter(click)
    top_click = nlargest(20, a, key = a.get)
    b = Counter(carts)
    top_carts = nlargest(20, b, key = b.get)
    c = Counter(order)
    top_order = nlargest(20, c, key = c.get)
    
    #append
    
    if  len(top_click) == 20 :
        preds.append(" ".join([str(id) for id in top_click]))
    
    elif len(top_click) < 20 :
        
        fillin_click = sample(top_click_C,20-len(top_click))
        
        top_click.extend(fillin_click)
        preds.append(" ".join([str(id) for id in top_click]))

        
    if  len(top_carts) == 20 :
        preds.append(" ".join([str(id) for id in top_carts]))
        
    elif len(top_carts) < 20 :
        
        fillin_carts = sample(top_carts_C,20-len(top_carts))
        
        top_carts.extend(fillin_carts)
        preds.append(" ".join([str(id) for id in top_carts]))                          

                      
    if  len(top_order) == 20 :
        preds.append(" ".join([str(id) for id in top_order]))

    elif len(top_order) < 20 :
        fillin_order = sample(top_order_C,20-len(top_order))
        
        top_order.extend(fillin_order)
        preds.append(" ".join([str(id) for id in top_order]))     
        
                              
    i = i+1

In [68]:
#preds

['59625 432679 1603001 1116095 709803 1700164 159789 1786822 1083299 1448648 202774 196279 448220 554660 1722991 1619041 954951 602469 865245 335674',
 '196279 263344 832192 1537102 1610178 636537 409620 654200 439859 608724 1116095 1062377 954951 1363864 1304792 634734 981335 709803 726458 339288',
 '1404100 526579 231894 198496 569954 383546 1082079 1112168 1051809 930005 1302234 1125519 409620 854275 230663 1407685 1610178 364101 1219795 768330']

In [118]:
# check on the len 3times of the input test data
len(preds)

5015409

In [119]:
sample_submission['labels'] = preds

In [120]:
sample_submission.to_csv('submission.csv', index=False)